---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [192]:
def answer_one():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel('Energy Indicators.xls')
    energy.drop(energy[[0,1]],axis=1,inplace=True)
    energy.drop(energy.index[243:],inplace=True)
    energy.drop(energy.index[:16],inplace=True)
    energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply']*=1000000
    energy=energy.set_index('Country')
    energy=energy.reset_index()
    #energy.reset_index(inplace=True)
    
    
    for ind,x in energy.iterrows():
        if(type(x['Energy Supply'])== type('Piy')):
            if(('...') in x['Energy Supply']):
                x['Energy Supply']=np.nan
                x['Energy Supply per Capita']=np.nan
                x['% Renewable']=np.nan
        if(type(x['Energy Supply per Capita'])== type('Piy')):
            if(('...') in x['Energy Supply per Capita']):
                x['Energy Supply']=np.nan
                x['Energy Supply per Capita']=np.nan
                x['% Renewable']=np.nan
        if(type(x['% Renewable'])== type('Piy')):
            if(('...') in x['% Renewable']):
                x['Energy Supply']=np.nan
                x['Energy Supply per Capita']=np.nan
                x['% Renewable']=np.nan
                
    energy['Country']=energy['Country'].apply(lambda x: x.split('(')[0].strip(' '))
    energy['Country']=energy['Country'].apply(lambda x: x.strip('0123456789'))
    energy.loc[:,'Country'].replace((["Republic of Korea","United States of America","United Kingdom of Great Britain and Northern Ireland","China, Hong Kong Special Administrative Region"]),["South Korea","United States","United Kingdom","Hong Kong"],inplace=True)
    
    col_list=[x for x in range(4,50)]
    
    gdp=pd.read_csv('world_bank.csv')
    gdp.columns=gdp.iloc[3]
    gdp.drop(gdp.index[:4],inplace=True)
    for i in col_list:
        gdp.drop(gdp.columns[[4]],axis=1,inplace=True)
    gdp.drop(gdp.columns[[1,2,3]],axis=1,inplace=True)
    
    count_list=["Korea, Rep.","Iran, Islamic Rep.","Hong Kong SAR, China"]
    to_be_replaced=["South Korea","Iran", "Hong Kong"]
           
    gdp.loc[gdp['Country Name']=='Korea, Rep.','Country Name']='South Korea'
    gdp.loc[gdp['Country Name']=='Iran, Islamic Rep.','Country Name']='Iran'
    gdp.loc[gdp['Country Name']=='Hong Kong SAR, China','Country Name']='Hong Kong'
    
    scim=pd.read_excel('scimagojr-3.xlsx')
    scim.sort('Rank')
    
    #print( (len(scim)+len(gdp)+len(energy)))
    scim=scim[scim['Rank']<=15]
    
    
    x=pd.merge(scim,energy,how='inner',left_on='Country',right_on='Country')
    y=pd.merge(x,gdp,how='inner',left_on='Country',right_on='Country Name')
    
   
    y.drop('Country Name',axis=1,inplace=True)
    
    
    y.reset_index()
    y=y.set_index('Country')
    
    y.index.name=None
    y.columns= ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    
    print(len(y))
    return y
answer_one()

15


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Rank  Documents  Citable documents  Citations  \
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       8896               8819      57470   
Australia             14       8831               8725      90765   
Brazil                15       8668               8596      60702   

                    Self-citations  Citations per document  H index  \
China                       411683                    4.70      138   
United States               265436                    8.20      230   
Japan                        61554                    7.31      134   
United Kingdom               37874                    9.84      139   
Russian Federation           12422                    1.85       57   
Canada                       40930                   12.01      149   
Germany                      27426                    8.26      126   
India                        37209                    8.58      115   
France                       28601                    9.93      114   
South Korea                  22595                    9.57      104   
Italy                        26661                   10.20      106   
Spain                        23964                   13.08      115   
Iran                         19125                    6.46       72   
Australia                    15606                   10.28      107   
Brazil                       14396                    7.00       86   

                   Energy Supply Energy Supply per Capita % Renewable  \
China               127191000000                       93     19.7549   
United States        90838000000                      286      11.571   
Japan                18984000000                      149     10.2328   
United Kingdom        7920000000                      124     10.6005   
Russian Federation   30709000000                      214     17.2887   
Canada               10431000000                      296     61.9454   
Germany              13261000000                      165     17.9015   
India                33195000000                       26     14.9691   
France               10597000000                      166     17.0203   
South Korea          11007000000                      221     2.27935   
Italy                 6530000000                      109     33.6672   
Spain                 4923000000                      106     37.9686   
Iran                  9172000000                      119     5.70772   
Australia             5386000000                      231     11.8108   
Brazil               12149000000                       59      69.648   

                            2006          2007          2008          2009  \
China               3.992331e+12  4.559041e+12  4.997775e+12  5.459247e+12   
United States       1.479230e+13  1.505540e+13  1.501149e+13  1.459484e+13   
Japan               5.496542e+12  5.617036e+12  5.558527e+12  5.251308e+12   
United Kingdom      2.419631e+12  2.482203e+12  2.470614e+12  2.367048e+12   
Russian Federation  1.385793e+12  1.504071e+12  1.583004e+12  1.459199e+12   
Canada              1.564469e+12  1.596740e+12  1.612713e+12  1.565145e+12   
Germany             3.332891e+12  3.441561e+12  3.478809e+12  3

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [28]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [193]:
def answer_two():
    
    return (156)
answer_two()

156

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [30]:
def answer_three():
    Top15 = answer_one()
    list_col=['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15['sum']=(Top15[list_col].sum(axis=1))/(10-Top15[list_col].isnull().sum(axis=1))
    Top15.sort('sum',ascending=False)
    return Top15['sum']
answer_three()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


China                 6.348609e+12
United States         1.536434e+13
Japan                 5.542208e+12
United Kingdom        2.487907e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
Germany               3.493025e+12
India                 1.769297e+12
France                2.681725e+12
South Korea           1.106715e+12
Italy                 2.120175e+12
Spain                 1.418078e+12
Iran                  4.441558e+11
Australia             1.164043e+12
Brazil                2.189794e+12
Name: sum, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [31]:
def answer_four():
    Top15 = answer_one()
    list_col=['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15['sum']=(Top15[list_col].sum(axis=1))/(10-Top15[list_col].isnull().sum(axis=1))
    num=(10-Top15[list_col].isnull().sum(axis=1))
    Top15.sort('sum',ascending=False,inplace=True)
    return (Top15['2015'][5]- Top15['2006'][5])
answer_four()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


246702696075.3999

### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [32]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].sum()/15
answer_five()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [33]:
def answer_six():
    Top15 = answer_one()
    maximum=max(Top15['% Renewable'])
    x=Top15[Top15['% Renewable']==maximum]
    y=(x.index.tolist()[0],)+tuple(x['% Renewable'])
    return y
answer_six()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


('Brazil', 69.64803)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [34]:
def answer_seven():
    Top15 = answer_one()
    Top15['ratio']=Top15['Self-citations']/Top15['Citations']
    maximum=max(Top15['ratio'])
    temp=Top15[Top15['ratio']==maximum]
    return ((temp.index.tolist()[0],)+tuple(temp['ratio']))
answer_seven()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [35]:
def answer_eight():
    Top15 = answer_one()
    Top15['pop']= Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15.sort('pop',ascending=False,inplace=True)
    return Top15.index.tolist()[2]
answer_eight()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [164]:
def answer_nine():
    import pandas as pd
    Top15 = answer_one()
    Top15['pop']= Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['ratio']=Top15['Citable documents']/Top15['pop']
    
    #r=np.cov('Energy Supply per Capita','ratio')
    #beta=((Top15.corr()*Top15['Energy Supply per Capita'].std()* Top15['ratio'].std())/(Top15['ratio'].var())).ix[0,1]
    #return (Top15['ratio'].corr(Top15['Energy Supply per Capita'],method='pearson'))
    
    #print (Top15[['ratio','Energy Supply per Capita']].corr())
    #Top15.corr()['ratio']['Energy Supply per Capita']
    #print(Top15)
    
    
    #print((Top15['ratio']))
    #print((Top15['Energy Supply per Capita']))
    
    #x=Top15.corr(method='pearson')
    #print(x)
    Top15=Top15.apply(pd.to_numeric, errors='coerce')
    return (Top15['ratio'].corr(Top15['Energy Supply per Capita']))
answer_nine()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0.79400104354429435

In [138]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [40]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [41]:
def answer_ten():
    import pandas as pd
    Top15 = answer_one()
    med=Top15['% Renewable'].median()
    Top15[Top15['% Renewable']>= med]['bool']=1
    Top15[Top15['% Renewable']< med]['bool']=0
    rnk=Top15['Rank'].tolist()
    ind=Top15.index.tolist()
    s=pd.Series(rnk,index=ind)
    s.sort('Rank')
    return s
answer_ten()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


China                  1
United States          2
Japan                  3
United Kingdom         4
Russian Federation     5
Canada                 6
Germany                7
India                  8
France                 9
South Korea           10
Italy                 11
Spain                 12
Iran                  13
Australia             14
Brazil                15
dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [49]:
def answer_eleven():
    import pandas as pd
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    diction ={    'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    ind=['Asia', 'Australia', 'Europe', 'North America', 'South America']
    col=['size', 'sum', 'mean', 'std']
    
    df=pd.DataFrame(index=ind,columns=col)
    df=df.fillna(0.0)
    for i,j in diction.items():
        df.loc[j]['size']+=1
        df.loc[j]['sum']+=Top15.loc[i]['PopEst']
    df['mean']=df['sum']/df['size']
    
    for i,j in diction.items():
        #print(Top15.loc[i]['PopEst']-df.loc[j]['mean'])
        #print('std initial '+ df.loc[j]['std'].astype(str))
        df.loc[j]['std']+=abs(Top15.loc[i]['PopEst']-df.loc[j]['mean'])**2
        #print('std final '+ df.loc[j]['std'].astype(str))
    df['std']=df['std']/df['size']
    print(df)
    #print(df['sum'])
    return df
answer_eleven()

               size           sum          mean           std
Asia            5.0  2.898666e+09  5.797333e+08  3.689392e+17
Australia       1.0  2.331602e+07  2.331602e+07  0.000000e+00
Europe          6.0  4.579297e+08  7.632161e+07  1.000384e+15
North America   2.0  3.528552e+08  1.764276e+08  1.993398e+16
South America   1.0  2.059153e+08  2.059153e+08  0.000000e+00


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


size           sum          mean           std
Asia            5.0  2.898666e+09  5.797333e+08  3.689392e+17
Australia       1.0  2.331602e+07  2.331602e+07  0.000000e+00
Europe          6.0  4.579297e+08  7.632161e+07  1.000384e+15
North America   2.0  3.528552e+08  1.764276e+08  1.993398e+16
South America   1.0  2.059153e+08  2.059153e+08  0.000000e+00

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [183]:
def answer_twelve():
    import pandas as pd
    import numpy as np
    Top15 = answer_one()
    diction ={    'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    #Top15['% Renewable']=pd.cut(Top15['% Renewable'],5,labels=False,precision=1)
    Top15['% Renewable']=pd.cut(Top15['% Renewable'],5)    
    indexes=Top15.index.tolist()
    for i in range(0,len(indexes)):
        indexes[i]=diction[indexes[i]]
    Top15['continent']=indexes
    
    #grouped=Top15.groupby(['continent','% Renewable'],as_index=True)
    #f=grouped.agg(np.size).dropna()
    f=Top15.groupby(['continent','% Renewable']).size()
    val=f.index.tolist()
    s=pd.Series(val)
    
    return (f)
answer_twelve()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [136]:
def answer_thirteen():
    import pandas as pd
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    ind=Top15.index.tolist()
    val=Top15['PopEst'].tolist()
    for i in range(0,len(val)):
        val[i]="{:,}".format(val[i])
    s=pd.Series(val,index=ind)
    return s
answer_thirteen()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:53: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!